# 1. 크롤링

### 준비 과정

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import re
import datetime
import arrow

In [2]:
driver = Chrome()

In [3]:
url = 'https://www.youtube.com/'
driver.get(url)

### 크롤링 for문

In [140]:
import pandas as pd
you=pd.read_excel("팀팀_유튜버목록.xlsx")
you.head()

,유튜버
0,RISABAE
1,PONY Syndrome
2,areumsongee아름송이
3,ssin 씬님
4,lamuqe


In [ ]:
final = []

In [39]:
## 재생목록 크롤링
for i in range(21,len(you)):
    search = driver.find_element_by_id("search")
    search.send_keys(you.유튜버[i]) #유튜버 입력
    time.sleep(5)
    button = driver.find_element_by_id("search-icon-legacy")
    button.click() #검색버튼 클릭

    time.sleep(5)
    channel = driver.find_element_by_id("channel-title")
    channel.click() #유튜버 채널 클릭

    time.sleep(5)
    menu  = driver.find_elements_by_css_selector('div.tab-content.style-scope.paper-tab')
    menu[2].click() #재생목록 클릭

    time.sleep(5)
    menu_list = driver.find_elements_by_css_selector('#view-more > a') #재생목록 리스트
   
    urls=[] #재생목록 url 리스트
    for url in menu_list:
        urls.append(url.get_attribute('href'))
    
    for i in range(0,len(urls)): 
        driver.get(urls[i]) #몇 번째 재생목록 접속
        titles = driver.find_elements_by_css_selector('a.yt-simple-endpoint.style-scope.ytd-playlist-video-renderer') 
        # 몇 번째 재생목록의 영상들    
        
        title_urls=[]
        for title_url in titles:
             title_urls.append(title_url.get_attribute('href')) #몇 번째 재생목록의 영상들 url 리스트            
            
        for i in range(0,len(title_urls)):
            try:
                driver.get(title_urls[i]) # 접속
                time.sleep(8)

                name = driver.find_element_by_css_selector('#owner-name > a').text
                contents = driver.find_element_by_css_selector('#header-description > h3 > yt-formatted-string > a').text #카테고리 이름
                titleName = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text #게시글 제목
                like = driver.find_elements_by_css_selector('yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text')[0].text #좋아요
                unlike = driver.find_elements_by_css_selector('yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text')[1].text #싫어요
                date = driver.find_element_by_css_selector('#upload-info > span').text #날짜
                click_number = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text
                
                 # 더보기란 수집
                more_button_text = '<paper-button id="more" aria-expanded="false" noink="" class="style-scope ytd-expander" role="button" tabindex="0" animated="" elevation="0" aria-disabled="false" hidden="">'
                if  more_button_text not in driver.page_source:
                    more_button = driver.find_elements_by_css_selector('#more > yt-formatted-string')
                    more_button[0].click() #더보기 클릭
                    more_contents = driver.find_element_by_css_selector('#description > yt-formatted-string').text #더보기란
                else:
                    more_contents = "없음"
                
                ## 댓글수 수집
                body = driver.find_element_by_tag_name("body")

                num_of_pagedowns = 10
                while num_of_pagedowns:
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(0.5)
                    num_of_pagedowns -= 1
                    try:
                        time.sleep(0.5)
                        reply = driver.find_element_by_css_selector('#count > yt-formatted-string').text
                    except:
                        None              

                final.append((name, contents, titleName,date, click_number, like, unlike, reply,more_contents))
            except NoSuchElementException:
                continue
    continue

In [40]:
# 데이터프레임으로 변환
df = pd.DataFrame(final, columns=['유튜버이름', '카테고리', '제목','날짜','조회수','좋아요','싫어요','댓글수' ,'더보기란'])
df.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,Made in Mia 미아,#왔다지름신 #꼼꼼리뷰,이거면 체육대회&페스티벌 인싸각? 이니스프리 체육대회에디션⛹🏻‍♀️,게시일: 2019. 5. 9.,"조회수 83,188회",2.3천,34,댓글 199개,#왔다지름신 #이니스프리 #체육대회에디션\ninstagram @hermia24
1,Made in Mia 미아,#왔다지름신 #꼼꼼리뷰,귀염뽀쨕! 릴리바이레드 에스더버니랜드 컬렉션💕#왔다지름신,게시일: 2019. 4. 30.,"조회수 51,169회",1.5천,14,댓글 165개,instagram @hermia24\n#왔다지름신 #릴리바이레드 #에스더러브스유\n...
2,Made in Mia 미아,#왔다지름신 #꼼꼼리뷰,봄옷하울🌸이라쓰고 지름신이라 읽는다!! #OOTD #쇼핑몰추천,게시일: 2019. 4. 20.,"조회수 50,917회",1.4천,6,댓글 191개,"instagram @hermia24\n#봄옷하울 #ootd #미아룩\n\n안녕, 라..."
3,Made in Mia 미아,#왔다지름신 #꼼꼼리뷰,"맥시멀리스트의 택배만 뜯는 브이로그 (옷, 화장품, 소품, 악세사리 등등)",게시일: 2019. 4. 13.,"조회수 62,953회",1.4천,19,댓글 173개,instagram @hermia24\n#택배가왔다 #프로소비러 #미아\n\n\n안녕...
4,Made in Mia 미아,#왔다지름신 #꼼꼼리뷰,봄맞이 대털이! 올리브영갔다가 40만원 넘게 쓰고 옴,게시일: 2019. 4. 5.,"조회수 164,508회",2.8천,72,댓글 295개,instagram @hermia24\n#올리브영 #왔다지름신 #미아\n*본 컨텐츠는...


In [41]:
# 데이터프레임 저장
df.to_excel('team_youtuber_미아부터 끝까지.xlsx', header=True)

In [82]:
# 데이터프레임 열기
import pandas as pd

df = pd.read_csv('team_youtuber_final.csv', engine='python')
df.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,게시일: 2017. 9. 19.,"조회수 668,372회",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,게시일: 2017. 9. 4.,"조회수 4,576,676회",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,게시일: 2017. 8. 23.,"조회수 677,521회",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,게시일: 2017. 8. 17.,"조회수 488,431회",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,게시일: 2017. 8. 14.,"조회수 347,425회",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


### 설문지 유튜버만 수집하기^^

In [4]:
main_you = pd.read_excel("팀팀_메인유튜버목록.xlsx")
main_you.head()

,main
0,RISABAE
1,조효진 Hyojin


In [5]:
final=[]

In [8]:
# 해보기..

standard_date = arrow.get('2018-06-30') #기준 날짜

search = driver.find_element_by_id("search")
search.send_keys(main_you.main[0]) #유튜버 입력
time.sleep(3)
button = driver.find_element_by_id("search-icon-legacy")
button.click() #검색버튼 클릭

time.sleep(5)
channel = driver.find_element_by_id("channel-title")
video_count = driver.find_element_by_css_selector('#video-count').text.split(' ')[1][:-1] #동영상 개수
channel.click() #유튜버 채널 클릭

time.sleep(5)
menu  = driver.find_elements_by_css_selector('div.tab-content.style-scope.paper-tab')
menu[1].click() #동영상 클릭

time.sleep(3)
menu_button = driver.find_element_by_css_selector('#play-all > ytd-button-renderer > a') 
menu_button.click() # 모두 재생 클릭

time.sleep(5)
driver.find_element_by_css_selector('#header-description > h3 > yt-formatted-string > a').click() #재생목록 클릭

time.sleep(3)
video_list = driver.find_elements_by_css_selector('#content > a') #동영상 리스트
video_url_list = []
for video in video_list:
    video_url_list.append(video.get_attribute('href')) #동영상 url 리스트(1-100)

for i in range(0, len(video_url_list)):
    driver.get(video_url_list[i]) 
    time.sleep(5)
    
    #날짜 작업
    date = driver.find_element_by_css_selector('#upload-info > span').text # 날짜
    date_text = re.match(r'.*: (.*). (.*). (.*).', date) #날짜만 가져오기
    date_text_bind = date_text.groups()[0] + "-" + date_text.groups()[1] + "-" + date_text.groups()[2] #날짜로 결합
    date_transfer = datetime.datetime.strptime(date_text_bind, "%Y-%m-%d") #날짜 객체로 전환
    video_date = arrow.get(date_transfer) #영상올린 날짜

    difference = video_date - standard_date #차이
    
    if difference.days < 0:
        break
    else:
        try:
            name = driver.find_element_by_css_selector('#owner-name > a').text
            contents = driver.find_element_by_css_selector('#header-description > h3 > yt-formatted-string > a').text #카테고리 이름
            titleName = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text #게시글 제목
            like = driver.find_elements_by_css_selector('yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text')[0].text #좋아요
            unlike = driver.find_elements_by_css_selector('yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text')[1].text #싫어요
            date = driver.find_element_by_css_selector('#upload-info > span').text #날짜
            click_number = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text

            # 더보기란 수집
            more_button_text = '<paper-button id="more" aria-expanded="false" noink="" class="style-scope ytd-expander" role="button" tabindex="0" animated="" elevation="0" aria-disabled="false" hidden="">'
            if  more_button_text not in driver.page_source:
                more_button = driver.find_elements_by_css_selector('#more > yt-formatted-string')
                more_button[0].click() #더보기 클릭
                more_contents = driver.find_element_by_css_selector('#description > yt-formatted-string').text #더보기란
            else:
                more_contents = "없음"

            ## 댓글수 수집
            body = driver.find_element_by_tag_name("body")

            num_of_pagedowns = 10
            while num_of_pagedowns:
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.5)
                num_of_pagedowns -= 1
                try:
                    time.sleep(0.5)
                    reply = driver.find_element_by_css_selector('#count > yt-formatted-string').text
                except:
                    None              

            final.append((name, contents, titleName,date, click_number, like, unlike, reply,more_contents))
        except NoSuchElementException:
            continue

In [10]:
df = pd.DataFrame(final, columns=['유튜버이름', '카테고리', '제목','날짜','조회수','좋아요','싫어요','댓글수' ,'더보기란'])
df.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,Uploads from Risabae,[Eng] 속눈썹 가장 쉽고 자연스럽게 붙이기+속눈썹 재사용법💫 l 이사배(RISA...,게시일: 2019. 5. 17.,"조회수 177,260회",4.7천,63,댓글 331개,#이사배속눈썹 #투디어속눈썹 #속눈썹재사용\n\n[제품 정보]\n\n#인조 속눈썹\...
1,RISABAE,Uploads from Risabae,[Eng] 인생 여권사진 건지는 메이크업 📸👩🏻✨ l 이사배(RISABAE Makeup),게시일: 2019. 5. 14.,"조회수 172,630회",4.2천,48,댓글 232개,#여권사진메이크업 #증명사진메이크업 #이사배증명사진\n\n[제품정보]\n\n#베이스...
2,RISABAE,Uploads from Risabae,[Eng] 뉴버전✨가성비 최고 브러시 10개 추천👍🏻 l 이사배(RISABAE Ma...,게시일: 2019. 5. 10.,"조회수 281,204회",8.1천,59,댓글 311개,"[제품 정보]\n\n\n1 필리밀리 - 빅 팬 브러시 851 / 13,000원\n..."
3,RISABAE,Uploads from Risabae,[Eng] 올로드샵 고양이상 데일리 메이크업😺 (아이라인 진짜 쉽게 그리는 꿀팁!)...,게시일: 2019. 5. 3.,"조회수 358,866회",8.5천,79,댓글 600개,#올로드샵 #고양이상메이크업 #아이라인\n\n[제품 정보]\n\n#렌즈\n- 디어렌...
4,RISABAE,Uploads from Risabae,11일간의 패션룩북 _ 렌즈&립정보 Fashion Lookbook l 이사배(RIS...,게시일: 2019. 4. 26.,"조회수 117,894회",2.7천,48,댓글 600개,#이사배룩북 #lookbook #이사배패션\n\n\n[룩북 정보]\n\n\n#LO...


In [11]:
# 데이터프레임 저장
df.to_excel('team_youtuber_이사배.xlsx', header=True)

# 여기까지

# 2. 데이터 전처리

### 유튜버 목록에 있는 것만 가져오기

In [83]:
final_you = []

In [84]:
for i in range(0, len(df)):
    for j in range(0,len(you)):
        if df.iloc[i,0] == you.iloc[j,0]:
            final_you.append(df.iloc[i,])
        else:
            continue

In [85]:
uni=pd.DataFrame(final_you, columns=['유튜버이름', '카테고리', '제목','날짜','조회수','좋아요','싫어요','댓글수' ,'더보기란'], )
uni.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,게시일: 2017. 9. 19.,"조회수 668,372회",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,게시일: 2017. 9. 4.,"조회수 4,576,676회",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,게시일: 2017. 8. 23.,"조회수 677,521회",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,게시일: 2017. 8. 17.,"조회수 488,431회",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,게시일: 2017. 8. 14.,"조회수 347,425회",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [86]:
uni = uni.reset_index(drop=True)

In [87]:
uni2 = uni.copy()
uni2.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,게시일: 2017. 9. 19.,"조회수 668,372회",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,게시일: 2017. 9. 4.,"조회수 4,576,676회",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,게시일: 2017. 8. 23.,"조회수 677,521회",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,게시일: 2017. 8. 17.,"조회수 488,431회",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,게시일: 2017. 8. 14.,"조회수 347,425회",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


### 날짜 고치기

In [88]:
uni3 = uni2.copy()

In [89]:
uni3.날짜 = uni3.날짜.apply(lambda x: x.replace('최초 공개', '게시일'))
uni3.날짜 = uni3.날짜.apply(lambda x: x.replace('실시간 스트리밍 시작일', '게시일'))

In [90]:
uni3.날짜 = uni3.날짜.apply(lambda x: "".join(x.split(' ')[1:4]))
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017.9.19.,"조회수 668,372회",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017.9.4.,"조회수 4,576,676회",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017.8.23.,"조회수 677,521회",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017.8.17.,"조회수 488,431회",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017.8.14.,"조회수 347,425회",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [91]:
uni3.날짜 = pd.to_datetime(uni3['날짜'], format="%Y.%m.%d.")
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,"조회수 668,372회",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,"조회수 4,576,676회",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,"조회수 677,521회",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,"조회수 488,431회",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,"조회수 347,425회",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


### 조회수 고치기

In [92]:
uni3.조회수 = uni3.조회수.str[4:-1]
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,"668,372",7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,"4,576,676",10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,"677,521",9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,"488,431",1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,"347,425",5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [93]:
uni3.조회수 = uni3.조회수.apply(lambda x: x.replace(',',''))
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7.6천,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10만,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9.5천,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1만,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5.1천,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [94]:
uni3.조회수 = pd.to_numeric(uni3.조회수)
uni3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4693 entries, 0 to 4692
Data columns (total 9 columns):
유튜버이름    4693 non-null object
카테고리     4693 non-null object
제목       4693 non-null object
날짜       4693 non-null datetime64[ns]
조회수      4693 non-null int64
좋아요      4693 non-null object
싫어요      4693 non-null object
댓글수      4690 non-null object
더보기란     4686 non-null object
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 330.1+ KB


### 좋아요 고치기

In [96]:
uni3.좋아요 = uni3.좋아요.apply(lambda x: x.replace("천", '00'))
uni3.좋아요 = uni3.좋아요.apply(lambda x: x.replace("만", '000'))
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7.600,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9.500,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5.100,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [97]:
uni3.좋아요 = uni3.좋아요.apply(lambda x: x.replace(".", ''))
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [98]:
# 갯수 설정 안해놓은 거 제외하기
index_nan = []
for i, j in enumerate(uni3.좋아요):
    if uni3.iloc[i,].좋아요 != "좋아요":
        index_nan.append(i)
    else:
        continue

In [99]:
uni3 = uni3[uni3.index.isin(index_nan)]
uni3 = uni3.reset_index(drop=True)
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,1천,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [100]:
uni3.좋아요 = pd.to_numeric(uni3.좋아요)
uni3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678 entries, 0 to 4677
Data columns (total 9 columns):
유튜버이름    4678 non-null object
카테고리     4678 non-null object
제목       4678 non-null object
날짜       4678 non-null datetime64[ns]
조회수      4678 non-null int64
좋아요      4678 non-null int64
싫어요      4678 non-null object
댓글수      4675 non-null object
더보기란     4671 non-null object
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 329.0+ KB


### 싫어요 고치기

In [101]:
uni3.싫어요 = uni3.싫어요.apply(lambda x: x.replace("천", '00'))
uni3.싫어요 = uni3.싫어요.apply(lambda x: x.replace("만", '000'))
uni3.싫어요 = uni3.싫어요.apply(lambda x: x.replace(".", ''))
uni3.싫어요 = pd.to_numeric(uni3.싫어요)
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,댓글 499개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,100,"댓글 8,084개",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,댓글 767개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,댓글 747개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,댓글 354개,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [102]:
uni3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678 entries, 0 to 4677
Data columns (total 9 columns):
유튜버이름    4678 non-null object
카테고리     4678 non-null object
제목       4678 non-null object
날짜       4678 non-null datetime64[ns]
조회수      4678 non-null int64
좋아요      4678 non-null int64
싫어요      4678 non-null int64
댓글수      4675 non-null object
더보기란     4671 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 329.0+ KB


### 댓글수 고치기

In [103]:
uni3.댓글수 = uni3.댓글수.str[3:-1]
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,499,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,100,"8,084",이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,767,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,747,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,354,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [104]:
uni3.댓글수 = uni3.댓글수.astype(str).apply(lambda x: x.replace(',',''))
uni3.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,499,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,100,8084,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,767,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,747,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,354,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [113]:
uni3['댓글수'][3637] = '41'
uni3['댓글수'][3919] = '82'
uni3['댓글수'][4628] = '114'

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
uni3.댓글수 = pd.to_numeric(uni3.댓글수)
uni3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678 entries, 0 to 4677
Data columns (total 9 columns):
유튜버이름    4678 non-null object
카테고리     4678 non-null object
제목       4678 non-null object
날짜       4678 non-null datetime64[ns]
조회수      4678 non-null int64
좋아요      4678 non-null int64
싫어요      4678 non-null int64
댓글수      4678 non-null int64
더보기란     4671 non-null object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 329.0+ KB


### 더보기란 채우기

In [115]:
uni3.loc[uni3.더보기란.isnull()]

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
716,areumsongee아름송이,#꿀팁,환절기에도 피부 포기못해??!! 스킨케어 특급 추천템?? ㅣ아름송이 areumsongee,2017-11-06,46021,630,58,81,NaN
743,areumsongee아름송이,#꿀팁,환절기에도 피부 포기못해??!! 스킨케어 특급 추천템?? ㅣ아름송이 areumsongee,2017-11-06,46021,630,58,81,NaN
1831,조효진 Hyojin,조효진의 DayDay,"포니선배릠과의 만남?!, 쨔미쓰 오랜만에 만남, 조효진의 짠내나는 제품샷 촬영 과정...",2019-04-25,129401,2600,26,267,NaN
2907,헤이즐 Heizle,#헤.사.세 (헤이즐이 사는 세상) | The world of Heizle,"[다이아페스티벌] 헤이즐/ 로즈하/ 하늘 ""헤로하""무대 생방송 무편집 클릭!!",2018-08-18,57836,495,26,192,NaN
3265,Jella젤라,[Self Makeup],"[좁쌀,고름형 여드름,홍조] 피부 뒤집어진날 get ready with me 여드름...",2017-10-18,45964,637,13,59,NaN
3297,Jella젤라,[Daily Life],"[vlog] 한국의 정글 실내동물원 주렁주렁! 실내데이트,하남스타필드 // 젤라jella",2017-11-30,48989,537,27,77,NaN
3524,KIMDAX킴닥스,?? 뷰티 튜토리얼 l Beauty Tutorial,Eng) 메이크업아티스트 자격증 온 날ㅠㅠ '그리너리 무화과 메이크업' 같이 준비해...,2018-11-17,32896,846,34,119,NaN


In [117]:
uni3['더보기란'][716] = 'http://drlims.com/shop/item.php?it_id... \n닥터림스 에이베타 리커버크림 다음편에는 에이베타리커버크림을 실제로 사용해서 효과 본 분들을 직접만나기로했어요:)\n 다음편을 기대하세요! \n★아름송이의 다른 일상도 궁금하다면! \n페이스북 : www.facebook.com/herseh \n인스타그램 : www.instagram.com/areumsongee'
uni3['더보기란'][743] = 'http://drlims.com/shop/item.php?it_id... \n닥터림스 에이베타 리커버크림 다음편에는 에이베타리커버크림을 실제로 사용해서 효과 본 분들을 직접만나기로했어요:)\n 다음편을 기대하세요! \n★아름송이의 다른 일상도 궁금하다면! \n페이스북 : www.facebook.com/herseh \n인스타그램 : www.instagram.com/areumsongee'
uni3['더보기란'][1831] = '없음'
uni3['더보기란'][2907] = '없음'
uni3['더보기란'][3265] = 'https://soundcloud.com/ehrling/ehrlin...\n\n누구보다 빠르게 스포 받기!! 인스타로 오세요🤣❤️\nhttps://www.instagram.com/im_jella_/\n\n안녕하세요~ 젤라입니다!\n\n여러분~~ 저 피부 왕창 뒤집어 졌다가 ㅠㅠ 다시 회복 했어요!!\n화장 독은.. 너무너무 무서운.. ㅠㅠ.....\n한번 오르면 정말 하루하루 너무 다르게 심해 지더라구요!\n\n화농성 여드름은 염증성으로 빨갛게 올라오는데,\n이건 고름이 먼저 차고 염증성으로 바뀌는.. 정말 답이 없는 트러블...\n\n다행히 일주일? 조금 넘어서 다 잡았고 지금 10프로만 더 회복하면 될 것 같아요 ㅎㅎ\n\n여러분도 화장독 조심하세요 ㅠㅠ!!\n\n시험 화이팅!! 하시구요😭💕💕💕\n\n재밌게 보셨다면 구독하기,좋아요,알람까지! 꾹꾹 부탁드려요😳💕\n\n\n-제품 정보-  \n\n🌹렌즈\n▸ \n-오렌즈-  30,000원\n시크리스 3콘 내츄럴 그레이 \n▸ \n그래픽직경 13.1mm\n\n직경은 작지만 오묘한 느낌 원하시는분 강추! 눈도 엄청 편해요~\n\n🌹메이크업\n\n▸ \n- 스킨푸드 - 8,900원\n청포도 프레쉬 베이스 2호\n\n3일 사용해본 결과 트러블이 심해지지 않았어요!\n\n▸ \n-에스티로더- 55,000원\n더블웨어 레디언트 브론즈 쿠션 스틱\n▸ \n라이트 미디움\n\n\n▸\n에스쁘아 \n프로테일러 리퀴드 파운데이션 ex\n텐헬시 ,아이보리퓨어\n판매가 : 38,000원\n\n여드름 심할땐 꼭 이것만 사용! 제 피부에 잘 맞아요\n\n▸\n-에스쁘아- 10,000원\n스키니 픽스 블렌더\n\n스펀지는 클리오 하이드라가.. 개인적으로 최고!\n\n▸ \n- 바비브라운 - 38,000원\n크리미 컨실러 베이지\n\n컬러 별로 있다구~ 제 피부톤엔 살짝 밝고 붉은 감이 있어요!\n\n▸ \n- 에스티로더- 55,000원\n더블웨어 브론즈 쿠션스틱\n\n어두운 피부에 컬러가 딱! 이에요.\n\n▸ \n- 스킨푸드 - 6,900원\n피치뽀송 멀티 피니시 파우더\n\n제 극건성 피부가 처음으로 편안함을 느낀 파우더 !\n\n▸\n- 맥 - 27,000원\n클리오아이 섀도우 소바\n\n워낙 유명한 국민 섀도우!\n코 쉐이딩 용으로 많이 쓴다던데.. 저랑 안맞아요.\n\n▸ \n- 멕 - 25,000원\n아이브로우 펜슬 링거링\n\n음... 좋은건 잘 모르겠어요.. 로드샵 제품도 괜찮은듯..\n\n▸\n- 맥 - 47,000원\n미네랄 라이즈 스킨피니쉬 치키브론즈\n\n가루날림 ㅠㅠ... 하지만 광은 예뻐요!\n\n▸ \n- 세잔느 - 13,000원\n네추럴 치크 블러셔 05호\n\n요즘 가장 많이 사용하는 립 ! 하나만 바르면 쌩얼 연출 가능! 립위에 플럼 효과를 내주고 입술을 촉촉 하게~ 립밤 대용으로도 사용!\n\n\n▸ \n- 네이처리퍼블릭 - 출시전\n바이플라워 트리플 무스 틴트 09호\n\n아직 출시 전 인가보아요 ㅠㅠ!! 입술 각질 제로! 딱 밀착되는 말린장미 끝판왕 꼭 사세요!!\n\n▸ \n- 네이처리퍼블릭 - 출시전\n바이 플라워 아이 섀도 18호\n\n발색 넘 잘 되고 텍스쳐도 깔끔~ 합니다! \n\n▸ \n- 네이처리퍼블릭 - 출시전\n바이 플라워 아이 섀도 23호\n\n그냥 제가 너무 좋아하는 컬러.. 젤라 메이크업에 빠지면 안되는!!!\n\n\n▸ \n네이처리퍼블릭 - 출시전\n바이 플라워 아이 섀도 03호\n\n저에겐 조금 허연~ 골드펄! 하얀 피부이신 분들 너무 영롱할 것 같아요!\n\n▸ \n- 네이처리퍼블릭 - 출시전 \n바이플라워 트리플 무스 틴트 09호 \n 무 조 건 사세요! 제 친구들도 미리 써보고 각질부각 전혀 없다고 난리에요!! \n\n -음악정보-\n\n[배경음] \n Title: Breeze \n Artist: Ehrling \n Genre: Dance & Electronic \n Mood: Happy \n Download: https://goo.gl/RFLFTc \n\n[오프닝곡] \n Galantis - Hunter (Selke Remix) \n Download: \n https://soundcloud.com/ehrling/ehrlin... \n\n [Contact mail] \n borabt@naver.com'
uni3['더보기란'][3297] = 'http://storefarm.naver.com/ban84/prod...\n남방 판매하는곳!!!!\n\n누구보다 빠르게 스포 받기!! 인 스타로 오세요🤣❤️\nhttps://www.instagram.com/im_jella_/\n\n안녕하세요~ 젤라입니다!\n\n여러분!! 동물농장 보시나요??\n저는 일요일 아침마다 봅니다!!!!\n\n이번 데이트는 정말 ㅠㅠ!!! 너무 행복했어요!!!\n소원을 이뤘고... (수달 손바닥 만지기)\n남치니도 좋아해 줘서 ㅠㅠ 진짜 대 성 공 ㅠㅠ!!\n\n바로 옆에 하남 스타필드도 있으니!! 하루 종일~ 따듯하게 데이트를 즐겨보세요~\n\n\n이번 영상도 재밌게, 유익하게 보셨다면 \n구독하기! 부탁드려요~😭💜\n\n---------------------------------------------------------------------------------\n\n[Contact mail]\njinbora78@naver.com'
uni3['더보기란'][3524] = "http://www.instagram.com/kimdax.daeun\n\n안녕하세요 여러분~\n커뮤니티에 올렸던 이야기죠...! 제가 얼마전 메이크업 국가 자격증에 합격했는데요, 꼼꼼한 준비 과정을 거친 덕에... 한번에 합격할 수 있었습니다 :)\n\n그래서 오늘은 시험에 대한 이야기, 합격한 썰과 자격증 배송 받은 날 했던 화장을 가져왔어요! 학교 가기 전 했던 늦가을 데일리 메이크업으로 좋은, '그리너리 무화과 메이크업'이에요. 브라운 톤의 립스틱과 차분한 톤의 색조를 이용한 가을 화장이에요. 재미있게 보셨길 바라요!\n\n항상 좋은 콘텐츠를 위한 킴닥스의 열정은 계속 됩니닷✨ㅋㅋ\n그럼 다음 영상에서도 만나요 우리!\n\n\n\n#메이크업아티스트 #메이크업국가자격증 #킴닥스\n\n\n\n\n+\n아 그리고 영상에서 언급한 쌤은 서울 강서구 화곡동에 #mauvelounge 샵을 내셨어요. 쌤 개인 샵이시고 이곳에서 국가 자격증 준비 반을 따로 운영하신다고 해요. 문의하실 분은 (010 2020 7292) 샵 번호로 '킴닥스님 구독스에요'라고 우리만의 암호명을 대주시면ㅋㅋ 친절히 상담해주실 거예요 :D\n\n______________________\n\n[사용제품 리스트]\n\n(스킨케어)\nThe Lab\nOligo hyaluronic acid 5000 toner\n\nBESONE\nCloud O2 cream\n\n\n(피부표현)\nDIOR\nDiorskinforever\nPerfecting makeup base\n\nDr.Jart+\nthe makeup fix on foundation\n01 pink light\n\nDr.Jart+\nthe makeup power balm concealer\n01 light\n\n\n(컨투어링)\nDior\ndior backstage\ncontouring palette\n001 universal\n\nMAC\nMineralize skinfinish\nlightscapade\n\nDior\ndior backstage\nGlow face palette\n001 universal\n\n\n(브로우)\nARITAUM\nmatte formula eyebrow pencil\n01 연갈색\n\nBENEFIT\nbrowzings\n03\n\nETUDE HOUSE\nColor my brow\n\n\n(아이메이크업)\nUrban decay\nEyeshadow primer potion\n\n\nNature Republic\npro touch color\nmaster shadow palette\n02\n\nSEPHORA\ncolorful\nFresh limonade N'276\n\n\nCLIO\nsharp so simple waterproof pencil liner\n02 brown\n\n\nARITAUM\nMermaid collection \nTwinkle eye glitter\n01\n\n\nMAYBELLINE\nthe magnum volume express\nmascara\nBlack\n\n\n\n(블러셔)\nCLINIQUE\ncheek pop\n05 nude pop\n\nMAC\nmineralize skinfinish\nMedium dark\n\n\n(립메이크업)\nAPIEU\ntrue velvet lipstick\nBR01 my Wednesday\n\nToo cool for school\nGlam lock \n04 demure\n\n\n\n_________\n\n렌즈 x\n\n브러시\n\n눈썹 Bobbi Brown\n섀도우 블렌딩 브러시 picasso 207 (강추)\n턱선 섀딩 브러시 too cool for school\n이외 모두 rebonne\n\n\n 스폰지 olive young"

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda

### 저장

In [118]:
# 데이터프레임 저장
uni3.to_excel('team_youtuber_전처리완료.xlsx', header=True)

C:\Users\USER\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'http://www.instagram.com/kimdax.daeun

안녕하세요%20여러분~
커뮤니티에%20올렸던%20이야기죠...!%20제가%20얼마전%20메이크업%20국가%20자격증에%20합격했는데요,%20꼼꼼한%20준비%20과정을%20거친%20덕에...%20한번에%20합격할%20수%20있었습니다%20:)

그래서%20오늘은%20시험에%20대한%20이야기,%20합격한%20썰과%20자격증%20배송%20받은%20날%20했던%20화장을%20가져왔어요!%20학교%20가기%20전%20했던%20늦가을%20데일리%20메이크업으로%20좋은,%20'그리너리%20무화과%20메이크업'이에요.%20브라운%20톤의%20립스틱과%20차분한%20톤의%20색조를%20이용한%20가을%20화장이에요.%20재미있게%20보셨길%20바라요!

항상%20좋은%20콘텐츠를%20위한%20킴닥스의%20열정은%20계속%20됩니닷✨ㅋㅋ
그럼%20다음%20영상에서도%20만나요%20우리!



' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


In [125]:
# 최종 파일
fin = pd.read_csv('team_youtuber_전처리완료.csv', engine='python')
fin.head()

,유튜버이름,카테고리,제목,날짜,조회수,좋아요,싫어요,댓글수,더보기란
0,RISABAE,# Make up Tutorials,현아 베베 메이크업 HyunA_BABE Makeup Tutorial l 이사배(Ri...,2017-09-19,668372,7600,142,499,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
1,RISABAE,# Make up Tutorials,[Eng/Thai]선미 가시나 메이크업 SUNMI - Gashina Cover Ma...,2017-09-04,4576676,10000,100,8084,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
2,RISABAE,# Make up Tutorials,[Eng/Thai] 꽃집아가씨 분위기 메이크업 Florist Inspired Mak...,2017-08-23,677521,9500,128,767,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
3,RISABAE,# Make up Tutorials,인어공주 애리얼 메이크업 ‘The Little Mermaid' Ariel Makeu...,2017-08-17,488431,1000,155,747,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...
4,RISABAE,# Make up Tutorials,[바캉스 메이크업 Summer Vacation Makeup Tutorial] l 이...,2017-08-14,347425,5100,90,354,이사배의 사배샵# 에 오신 것을 환영합니다!\n*매주 화/금/토 카카오TV 생방송*...


In [130]:
fin2 = fin.drop_duplicates()
fin2.to_excel('team_youtuber_전처리완료2.xlsx', header=True)

# 유튜버 선정

In [20]:
import pandas as pd
cre = pd.read_csv("설문조사 카테고리 정리.csv", engine = "python", encoding='cp949')
cre.head()

,미용피트니스,쇼핑,여행,"취미, 레저",홈 및 가든,도서 및 문학,유튜버목록
0,다영,한별,KINDACOOL,로즈하,kindacool,윤그린,다영
1,이사배,한별,유네린,이연,트위티,라뮤끄,이사배
2,이사배,잉화,오늘은 예림,에바,트위티,리안,이사배
3,라뮤끄,kindacool,오늘의 하늘,로즈하,NaN,지구데일리,라뮤끄
4,회사원A,씬님,에바,로즈하,NaN,NaN,회사원A


In [21]:
cre.유튜버목록.value_counts()

이사배               164
한별                 61
조효진                51
포니                 50
오늘의 하늘             50
씬님                 38
회사원A               30
우린                 27
새벽                 26
라뮤끄                23
헤이즐                22
에바                 20
홀리                 19
디렉터파이              18
유트루                17
로즈하                17
리안                 16
kindacool          13
연두콩                13
다영                 12
채소                 10
sydneytoyou         9
Minjeong Park       8
데이지                 7
달님달링                7
디어제리                7
윤그린                 7
소윤                  7
애림                  7
김습습                 6
                 ... 
소울언니                1
sydenytoyou         1
세림                  1
호버                  1
집 팔아 화장품 사는 깡나      1
진주쌤                 1
뽐니                  1
쩡유                  1
함경식                 1
이승인                 1
슈스스                 1
썬지                  1
프롬핸                 1
김무비                 1
바라던바다     